# Fractional Translation Optimization

Implement tiny position adjustments (fractional translation) to find improvements that SA misses.
This technique uses much smaller steps than SA and can find micro-improvements.

In [1]:
import sys
import os
os.chdir('/home/code/experiments/008_fractional_translation')
sys.path.insert(0, '/home/code')

import numpy as np
import pandas as pd
import json
import time
from shapely.geometry import Polygon
from shapely.ops import unary_union

from code.tree_geometry import TX, TY, calculate_score, get_tree_vertices_numba
from code.overlap_check import has_overlap
from code.utils import parse_submission, save_submission

print("Modules imported successfully")

Modules imported successfully


In [2]:
# Load baseline
baseline_df = pd.read_csv('/home/code/experiments/001_valid_baseline/submission.csv')
baseline_configs = parse_submission(baseline_df)

# Calculate baseline scores per N
baseline_scores = {}
for n in range(1, 201):
    baseline_scores[n] = calculate_score(baseline_configs[n])

baseline_total = sum(baseline_scores.values())
print(f"Baseline total score: {baseline_total:.6f}")

Baseline total score: 70.615102


In [3]:
def get_tree_polygon(x, y, angle_deg):
    """Get tree polygon at given position and angle."""
    rx, ry = get_tree_vertices_numba(x, y, angle_deg)
    return Polygon(zip(rx, ry))

def check_no_overlap_shapely(trees):
    """Check if any trees overlap using Shapely."""
    if len(trees) <= 1:
        return True
    
    polygons = [get_tree_polygon(x, y, a) for x, y, a in trees]
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                if not polygons[i].touches(polygons[j]):
                    inter = polygons[i].intersection(polygons[j])
                    if inter.area > 1e-12:
                        return False
    return True

def calculate_bbox_score(trees):
    """Calculate bounding box score for a set of trees."""
    if len(trees) == 0:
        return 0.0
    
    all_xs = []
    all_ys = []
    for x, y, angle in trees:
        rx, ry = get_tree_vertices_numba(x, y, angle)
        all_xs.extend(rx)
        all_ys.extend(ry)
    
    width = max(all_xs) - min(all_xs)
    height = max(all_ys) - min(all_ys)
    side = max(width, height)
    n = len(trees)
    return (side ** 2) / n

print("Helper functions defined")

Helper functions defined


In [4]:
def fractional_translation_single_n(trees, frac_steps, directions, max_iterations=3):
    """
    Apply fractional translation to a single N configuration.
    Returns improved configuration and improvement amount.
    """
    n = len(trees)
    current_trees = [list(t) for t in trees]  # Make mutable copies
    current_score = calculate_bbox_score(current_trees)
    
    total_improvement = 0.0
    
    for iteration in range(max_iterations):
        improved_this_iteration = False
        
        for tree_idx in range(n):
            for step in frac_steps:
                for dx, dy in directions:
                    # Try moving tree
                    new_trees = [list(t) for t in current_trees]
                    new_trees[tree_idx][0] += dx * step
                    new_trees[tree_idx][1] += dy * step
                    
                    # Check validity
                    if not check_no_overlap_shapely(new_trees):
                        continue
                    
                    # Check score
                    new_score = calculate_bbox_score(new_trees)
                    
                    if new_score < current_score - 1e-12:
                        improvement = current_score - new_score
                        current_trees = new_trees
                        current_score = new_score
                        total_improvement += improvement
                        improved_this_iteration = True
        
        if not improved_this_iteration:
            break
    
    return [(t[0], t[1], t[2]) for t in current_trees], total_improvement

print("Fractional translation function defined")

Fractional translation function defined


In [5]:
# Define fractional steps and directions
frac_steps = [0.001, 0.0005, 0.0002, 0.0001, 0.00005, 0.00002, 0.00001]
directions = [(0,1), (0,-1), (1,0), (-1,0), (1,1), (1,-1), (-1,1), (-1,-1)]

print(f"Fractional steps: {frac_steps}")
print(f"Directions: {directions}")
print(f"Total moves per tree per iteration: {len(frac_steps) * len(directions)} = {len(frac_steps) * len(directions)}")

Fractional steps: [0.001, 0.0005, 0.0002, 0.0001, 5e-05, 2e-05, 1e-05]
Directions: [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
Total moves per tree per iteration: 56 = 56


In [6]:
# Test on a few N values first
test_n_values = [10, 20, 50, 100]

print("Testing fractional translation on selected N values...")
print("="*70)

test_results = {}
for n in test_n_values:
    print(f"\nN={n}:")
    start_time = time.time()
    
    original_score = baseline_scores[n]
    improved_trees, improvement = fractional_translation_single_n(
        baseline_configs[n], frac_steps, directions, max_iterations=2
    )
    
    elapsed = time.time() - start_time
    new_score = calculate_bbox_score(improved_trees)
    
    test_results[n] = {
        'original': original_score,
        'new': new_score,
        'improvement': improvement,
        'trees': improved_trees
    }
    
    status = "✅ IMPROVED" if improvement > 1e-9 else "❌ No improvement"
    print(f"  Original: {original_score:.9f}")
    print(f"  New:      {new_score:.9f}")
    print(f"  Improvement: {improvement:.9f} {status}")
    print(f"  Time: {elapsed:.1f}s")

Testing fractional translation on selected N values...

N=10:


  Original: 0.376630041
  New:      0.376630041
  Improvement: 0.000000000 ❌ No improvement
  Time: 0.2s

N=20:


  Original: 0.376056860
  New:      0.376056860
  Improvement: 0.000000000 ❌ No improvement
  Time: 1.1s

N=50:


  Original: 0.360753134
  New:      0.360753134
  Improvement: 0.000000000 ❌ No improvement
  Time: 11.1s

N=100:


  Original: 0.343394816
  New:      0.343394724
  Improvement: 0.000000092 ✅ IMPROVED
  Time: 177.6s


In [ ]:
# Run on all N values with limited iterations for speed
print("\nRunning fractional translation on ALL N values...")
print("="*70)

improved_configs = {}
improvements = []

start_time = time.time()
for n in range(1, 201):
    original_score = baseline_scores[n]
    improved_trees, improvement = fractional_translation_single_n(
        baseline_configs[n], frac_steps, directions, max_iterations=1
    )
    
    improved_configs[n] = improved_trees
    
    if improvement > 1e-12:
        improvements.append((n, improvement))
        print(f"N={n}: Improved by {improvement:.9f}")

elapsed = time.time() - start_time
print(f"\nCompleted in {elapsed:.1f}s")
print(f"Total improvements found: {len(improvements)}")

In [ ]:
# Calculate new total score
new_total = sum(calculate_bbox_score(improved_configs[n]) for n in range(1, 201))

print(f"\nBaseline total: {baseline_total:.6f}")
print(f"New total: {new_total:.6f}")
print(f"Total improvement: {baseline_total - new_total:.9f}")

if improvements:
    print(f"\nTop improvements:")
    for n, imp in sorted(improvements, key=lambda x: -x[1])[:10]:
        print(f"  N={n}: {imp:.9f}")

In [ ]:
# Validate all configurations
print("\nValidating all configurations...")
invalid_n = []
for n in range(1, 201):
    if not check_no_overlap_shapely(improved_configs[n]):
        invalid_n.append(n)

if not invalid_n:
    print("✅ All configurations valid (no overlaps)")
else:
    print(f"❌ Invalid configurations: {invalid_n}")

In [ ]:
# Save submission
save_submission(improved_configs, 'submission.csv')
print("Saved submission.csv")

# Validate submission format
df = pd.read_csv('submission.csv')
print(f"Rows: {len(df)}")
print(f"Columns: {df.columns.tolist()}")

# Check for NaN values
for col in ['x', 'y', 'deg']:
    vals = df[col].astype(str).str.replace('s', '', regex=False)
    nan_count = vals.apply(lambda x: 'nan' in x.lower()).sum()
    if nan_count > 0:
        print(f"❌ WARNING: {nan_count} NaN values in {col}")
    else:
        print(f"✅ No NaN values in {col}")

In [ ]:
# Save metrics
metrics = {
    'cv_score': new_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - new_total,
    'num_improvements': len(improvements),
    'frac_steps': frac_steps,
    'directions': directions,
    'improvements': [(n, imp) for n, imp in improvements],
    'notes': 'Fractional translation with tiny position adjustments'
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nFinal CV Score: {new_total:.6f}")

In [ ]:
# Copy to submission folder
import shutil
shutil.copy('submission.csv', '/home/submission/submission.csv')
print("Copied submission to /home/submission/")